# 데이터 진단 및 문제 파악

분석 결과에서 "정의되지 않은 코드" 문제를 파악하고 해결합니다.

In [1]:
import sys
sys.path.append('C:/SKN_19/SKN19-2nd-2Team')

import pandas as pd
import numpy as np
from data.var_mapping import VAR_DICT, get_var_info

# 데이터 로드
df = pd.read_csv('../../data/analy_data.csv')
print(f"데이터 shape: {df.shape}")

데이터 shape: (89822, 210)


## 1. 각 변수의 실제 코드값 확인

In [2]:
# 분석 대상 변수
analysis_vars = ['sob_01z1', 'sob_02z1', 'soa_01z1', 'soa_06z2', 'soa_07z1', 'sod_02z3']

print("="*80)
print("📊 각 변수의 실제 코드값 분포")
print("="*80)

undefined_issues = []  # 문제 수집

for var in analysis_vars:
    print(f"\n### {var} - {get_var_info(var)['name']}")
    print("-" * 60)
    
    # 실제 데이터의 코드값
    actual_values = df[var].value_counts().sort_index()
    print("\n실제 데이터의 코드값:")
    print(actual_values)
    
    # VAR_DICT에 정의된 코드값
    var_info = get_var_info(var)
    if var_info and var_info.get('values'):
        defined_codes = set(var_info['values'].keys())
        print(f"\nVAR_DICT에 정의된 코드: {sorted(defined_codes)}")
        
        # 정의되지 않은 코드 찾기
        actual_codes = set(actual_values.index.dropna())
        undefined_codes = actual_codes - defined_codes
        
        if undefined_codes:
            print(f"\n⚠️ 정의되지 않은 코드: {sorted(undefined_codes)}")
            print(f"   해당 코드의 빈도:")
            for code in sorted(undefined_codes):
                count = actual_values.get(code, 0)
                pct = count / len(df) * 100
                print(f"     {code}: {count}건 ({pct:.2f}%)")
                undefined_issues.append({
                    '변수': var,
                    '변수명': var_info['name'],
                    '코드': code,
                    '건수': count,
                    '비율(%)': f"{pct:.2f}"
                })
        else:
            print("\n✅ 모든 코드가 정의되어 있음")
    else:
        print("\n⚠️ VAR_DICT에 values 정보 없음")
    
    print()

📊 각 변수의 실제 코드값 분포

### sob_01z1 - 교육수준(최종학력)
------------------------------------------------------------

실제 데이터의 코드값:
sob_01z1
1.0      1465
2.0        54
3.0      9897
4.0     10378
5.0     31369
6.0     11171
7.0     21721
8.0      3733
77.0       33
99.0        1
Name: count, dtype: int64

VAR_DICT에 정의된 코드: [1, 2, 3, 4, 5, 6, 7, 8]

⚠️ 정의되지 않은 코드: [77.0, 99.0]
   해당 코드의 빈도:
     77.0: 33건 (0.04%)
     99.0: 1건 (0.00%)


### sob_02z1 - 졸업상태
------------------------------------------------------------

실제 데이터의 코드값:
sob_02z1
1.0    78671
2.0      346
3.0     7377
4.0     1875
7.0       33
8.0     1519
9.0        1
Name: count, dtype: int64

VAR_DICT에 정의된 코드: [1, 2, 3, 4]

⚠️ 정의되지 않은 코드: [7.0, 8.0, 9.0]
   해당 코드의 빈도:
     7.0: 33건 (0.04%)
     8.0: 1519건 (1.69%)
     9.0: 1건 (0.00%)


### soa_01z1 - 경제활동 여부
------------------------------------------------------------

실제 데이터의 코드값:
soa_01z1
1.0    64823
2.0    24995
7.0        1
9.0        3
Name: count, dtype: int64

VAR_DICT에 정의된 코드:

## 2. churn 변수 확인

In [3]:
print("\n📊 타겟 변수 (churn) 분석")
print("="*60)
print("\n분포:")
print(df['churn'].value_counts())
print(f"\n결측값: {df['churn'].isna().sum()}건")
print(f"금연 성공률: {df['churn'].mean() * 100:.2f}%")

# churn이 있는 데이터만 필터링
df_with_churn = df[df['churn'].notna()]
print(f"\nchurn이 있는 데이터: {len(df_with_churn)}건 ({len(df_with_churn)/len(df)*100:.1f}%)")


📊 타겟 변수 (churn) 분석

분포:
churn
1    49251
0    40571
Name: count, dtype: int64

결측값: 0건
금연 성공률: 54.83%

churn이 있는 데이터: 89822건 (100.0%)


## 3. 변수별 결측값 및 특수코드 분석

In [4]:
print("\n📊 변수별 데이터 품질 분석")
print("="*80)

quality_report = []

for var in analysis_vars:
    var_info = get_var_info(var)
    
    # 결측값
    na_count = df[var].isna().sum()
    na_pct = na_count / len(df) * 100
    
    # 특수코드 (응답거부, 모름, 비해당 등)
    special_codes = []
    special_count = 0
    
    if var_info and var_info.get('values'):
        for code, label in var_info['values'].items():
            if any(keyword in str(label) for keyword in ['응답거부', '모름', '비해당']):
                special_codes.append(code)
                special_count += (df[var] == code).sum()
    
    special_pct = special_count / len(df) * 100
    
    # 유효 데이터
    valid_count = len(df) - na_count - special_count
    valid_pct = valid_count / len(df) * 100
    
    quality_report.append({
        '변수': var_info['name'] if var_info else var,
        '전체': len(df),
        '결측값': f"{na_count} ({na_pct:.1f}%)",
        '특수코드': f"{special_count} ({special_pct:.1f}%)",
        '유효데이터': f"{valid_count} ({valid_pct:.1f}%)"
    })

quality_df = pd.DataFrame(quality_report)
print(quality_df.to_string(index=False))


📊 변수별 데이터 품질 분석
        변수    전체      결측값     특수코드          유효데이터
교육수준(최종학력) 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)
      졸업상태 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)
   경제활동 여부 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)
 직업분류(재분류) 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)
    종사상 지위 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)
      혼인상태 89822 0 (0.0%) 0 (0.0%) 89822 (100.0%)


## 4. 교차 분석: churn과 함께 유효한 데이터

In [5]:
print("\n📊 churn과 함께 분석 가능한 데이터")
print("="*80)

churn_quality = []

for var in analysis_vars:
    var_info = get_var_info(var)
    
    # churn과 해당 변수 모두 유효한 데이터
    both_valid = df[(df['churn'].notna()) & (df[var].notna())]
    
    # 특수코드 제외
    if var_info and var_info.get('values'):
        special_codes = [
            code for code, label in var_info['values'].items()
            if any(keyword in str(label) for keyword in ['응답거부', '모름', '비해당'])
        ]
        both_valid_clean = both_valid[~both_valid[var].isin(special_codes)]
    else:
        both_valid_clean = both_valid
    
    churn_quality.append({
        '변수': var_info['name'] if var_info else var,
        '원본 데이터': len(df),
        '분석 가능': len(both_valid_clean),
        '비율(%)': f"{len(both_valid_clean)/len(df)*100:.1f}",
        '금연성공률(%)': f"{both_valid_clean['churn'].mean()*100:.1f}"
    })

churn_quality_df = pd.DataFrame(churn_quality)
print(churn_quality_df.to_string(index=False))


📊 churn과 함께 분석 가능한 데이터
        변수  원본 데이터  분석 가능 비율(%) 금연성공률(%)
교육수준(최종학력)   89822  89822 100.0     54.8
      졸업상태   89822  89822 100.0     54.8
   경제활동 여부   89822  89822 100.0     54.8
 직업분류(재분류)   89822  89822 100.0     54.8
    종사상 지위   89822  89822 100.0     54.8
      혼인상태   89822  89822 100.0     54.8


## 5. 진단 결과 요약

In [6]:
print("\n" + "="*80)
print("📋 진단 결과 요약 및 권장 조치")
print("="*80)

print("\n### 1️⃣ 정의되지 않은 코드값")
if undefined_issues:
    print(f"\n⚠️ 총 {len(undefined_issues)}개의 정의되지 않은 코드 발견")
    undefined_df = pd.DataFrame(undefined_issues)
    print("\n" + undefined_df.to_string(index=False))
    print("\n→ 조치: var_mapping.py에 위 코드값 추가 필요")
else:
    print("\n✅ 모든 코드값이 정의되어 있음")

print("\n### 2️⃣ 데이터 정제 방안")
print("\n✅ 권장: 특수코드(응답거부, 모름, 비해당) 제거")
print("✅ 권장: 결측값 제거")
print("⚠️ 주의: 제거 후 표본 크기 충분성 확인 필요")

print("\n### 3️⃣ 분석 가능 표본 크기")
min_sample = churn_quality_df['분석 가능'].min()
print(f"\n최소 표본 크기: {min_sample:,}건")

# 숫자로 변환하여 비교
if min_sample > 1000:
    print("✅ 충분한 표본 크기 확보")
elif min_sample > 500:
    print("⚠️ 표본 크기가 다소 작음. 해석 시 주의 필요")
else:
    print("❌ 표본 크기가 매우 작음. 통계적 검정력 부족 우려")

print("\n### 4️⃣ 다음 단계")
print("\n1. var_mapping.py 업데이트 (정의되지 않은 코드 추가)")
print("2. 데이터 정제 스크립트 실행")
print("3. 분석 재실행")
print("4. 특성 중요도 분석 추가")
print("5. 최종 해석 및 보고서 작성")

print("\n" + "="*80)
print("✅ 진단 완료!")
print("="*80)


📋 진단 결과 요약 및 권장 조치

### 1️⃣ 정의되지 않은 코드값

⚠️ 총 15개의 정의되지 않은 코드 발견

      변수        변수명   코드    건수 비율(%)
sob_01z1 교육수준(최종학력) 77.0    33  0.04
sob_01z1 교육수준(최종학력) 99.0     1  0.00
sob_02z1       졸업상태  7.0    33  0.04
sob_02z1       졸업상태  8.0  1519  1.69
sob_02z1       졸업상태  9.0     1  0.00
soa_01z1    경제활동 여부  7.0     1  0.00
soa_01z1    경제활동 여부  9.0     3  0.00
soa_06z2  직업분류(재분류) 77.0     9  0.01
soa_06z2  직업분류(재분류) 88.0 24995 27.83
soa_06z2  직업분류(재분류) 99.0     4  0.00
soa_07z1     종사상 지위  7.0     9  0.01
soa_07z1     종사상 지위  8.0 24995 27.83
soa_07z1     종사상 지위  9.0     4  0.00
sod_02z3       혼인상태  7.0    15  0.02
sod_02z3       혼인상태  9.0     2  0.00

→ 조치: var_mapping.py에 위 코드값 추가 필요

### 2️⃣ 데이터 정제 방안

✅ 권장: 특수코드(응답거부, 모름, 비해당) 제거
✅ 권장: 결측값 제거
⚠️ 주의: 제거 후 표본 크기 충분성 확인 필요

### 3️⃣ 분석 가능 표본 크기

최소 표본 크기: 89,822건
✅ 충분한 표본 크기 확보

### 4️⃣ 다음 단계

1. var_mapping.py 업데이트 (정의되지 않은 코드 추가)
2. 데이터 정제 스크립트 실행
3. 분석 재실행
4. 특성 중요도 분석 추가
5. 최종 해석 및 보고서 작성

✅ 진단 완료!


## 6. 체크리스트 출력

In [8]:
print("\n📋 체크리스트\n")
print("="*80)

print(f"\n✅ 데이터 기본 정보:")
print(f"   - 전체 데이터: {len(df):,}건")
print(f"   - churn 있는 데이터: {len(df_with_churn):,}건")
print(f"   - 전체 금연 성공률: {df['churn'].mean()*100:.2f}%")

print(f"\n✅ 정의되지 않은 코드:")
if undefined_issues:
    print(f"   ⚠️ {len(undefined_issues)}개 발견")
    for issue in undefined_issues:
        print(f"   - {issue['변수명']}: 코드 {issue['코드']} ({issue['건수']:,}건, {issue['비율(%)']}%)")
else:
    print("   ✅ 없음")

print(f"\n✅ 분석 가능 표본:")
print(f"   - 최소: {churn_quality_df['분석 가능'].min():,}건")
print(f"   - 최대: {churn_quality_df['분석 가능'].max():,}건")

print(f"\n✅ 특이사항:")
# 표본이 가장 작은 변수
min_idx = churn_quality_df['분석 가능'].idxmin()
print(f"   - 표본이 가장 작은 변수: {churn_quality_df.loc[min_idx, '변수']}")

print("\n" + "="*80)
print("💡 이 정보를 최적화된 분석 노트북을 생성합니다!")
print("="*80)


📋 체크리스트


✅ 데이터 기본 정보:
   - 전체 데이터: 89,822건
   - churn 있는 데이터: 89,822건
   - 전체 금연 성공률: 54.83%

✅ 정의되지 않은 코드:
   ⚠️ 15개 발견
   - 교육수준(최종학력): 코드 77.0 (33건, 0.04%)
   - 교육수준(최종학력): 코드 99.0 (1건, 0.00%)
   - 졸업상태: 코드 7.0 (33건, 0.04%)
   - 졸업상태: 코드 8.0 (1,519건, 1.69%)
   - 졸업상태: 코드 9.0 (1건, 0.00%)
   - 경제활동 여부: 코드 7.0 (1건, 0.00%)
   - 경제활동 여부: 코드 9.0 (3건, 0.00%)
   - 직업분류(재분류): 코드 77.0 (9건, 0.01%)
   - 직업분류(재분류): 코드 88.0 (24,995건, 27.83%)
   - 직업분류(재분류): 코드 99.0 (4건, 0.00%)
   - 종사상 지위: 코드 7.0 (9건, 0.01%)
   - 종사상 지위: 코드 8.0 (24,995건, 27.83%)
   - 종사상 지위: 코드 9.0 (4건, 0.00%)
   - 혼인상태: 코드 7.0 (15건, 0.02%)
   - 혼인상태: 코드 9.0 (2건, 0.00%)

✅ 분석 가능 표본:
   - 최소: 89,822건
   - 최대: 89,822건

✅ 특이사항:
   - 표본이 가장 작은 변수: 교육수준(최종학력)

💡 이 정보를 최적화된 분석 노트북을 생성합니다!
